# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **rostock**, Germany.

Since there are lots of restaurants in rostock we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of rostock center will be obtained using **Google Maps API geocoding** of well known rostock location (Alexanderplatz)

### Neighborhood Candidates

Let"s create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around rostock city center.

Let"s first find the latitude & longitude of rostock city center, using specific, well known address and Google Maps geocoding API.

In [65]:
import folium
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import geocoder
import pgeocode

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [66]:
# read csv file
df = pd.read_csv("data/DE.txt", delimiter="\t", names=["country-code", "postal-code", "city",
                 "state", "state-abbrev", "region", "r-code", "province", "p-code", "latitude", "longitude", "accuracy"], encoding="utf-8")

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values) == True]

# drop rows where province does not include Rostock
df = filter_rows_by_values(df, "province", ["Kreisfreie Stadt Rostock", "Landkreis Rostock"])

df.head()

# df.shape


,country-code,postal-code,city,state,state-abbrev,region,r-code,province,p-code,latitude,longitude,accuracy
6251,DE,17166,Groß Wokern,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7513,12.4925,4.0
6252,DE,17166,Bristow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7123,12.6257,4.0
6253,DE,17166,Dalkendorf,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8148,12.5021,4.0
6254,DE,17166,Bülow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.6899,12.6007,4.0
6255,DE,17166,Alt Sührkow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8024,12.6512,4.0


In [67]:
# get Rostock geo-location
address = "Rostock, Germany"
geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
rostock_center = [latitude, longitude]
print(
    f"The geograpical coordinates of Rostock Germnay are {latitude}, {longitude}.")


The geograpical coordinates of Rostock Germnay are 54.0924445, 12.1286127.


In [68]:
# create map
map_rostock = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, city in zip(df["latitude"], df["longitude"], df["city"]):
    label = f"{city}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#87cefa",
        fill_opacity=0.5,
        parse_html=False).add_to(map_rostock)
map_rostock


In [69]:
import pyproj
from pyproj import Transformer

import math


def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj="latlong", datum="WGS84")
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum="WGS84")
    transformer = Transformer.from_proj(proj_latlon, proj_xy)
    xy = transformer.transform(lon, lat)
    return xy[0], xy[1]


def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj="latlong", datum="WGS84")
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum="WGS84")
    transformer = Transformer.from_proj(proj_xy, proj_latlon)
    lonlat = transformer.transform(x, y)
    return lonlat[0], lonlat[1]


def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

# lonlat_to_xy(53.7513, 9.5665)
# xy_to_lonlat(5096991.4295275, 1350788.7431971882)


In [70]:
from constants import GOOGLE_API_KEY

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?key={api_key}&latlng={latitude},{longitude}"
        response = requests.get(url).json()
        if verbose:
            print("Google Maps API JSON result =>", response)
        results = response["results"]
        address = results[0]["formatted_address"]
        return address
    except:
        return None


addr = get_address(GOOGLE_API_KEY, rostock_center[0], rostock_center[1])
print("Reverse geocoding check")
print("-----------------------")
print(f"Address of [{rostock_center[0]}, {rostock_center[1]}] is: {addr}")


Reverse geocoding check
-----------------------
Address of [54.0924445, 12.1286127] is: Stadthafen 71, 18057 Rostock, Germany


In [71]:
print("Obtaining location addresses: ", end="")
addresses = []
for lat, lon in zip(df["latitude"], df["longitude"]):
    address = get_address(GOOGLE_API_KEY, lat, lon)
    if address is None:
        address = "NO ADDRESS"
    # We don"t need country part of address
    address = address.replace(", Germany", "")
    addresses.append(address)
    print(" .", end="")
print(" done.")


Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [77]:
addresses[10:20]

['Dorfstraße 64, 17168 Groß Wüstenfelde',
 'Poggelow 16, 17168 Schwasdorf',
 'Unnamed Road, 17168 Sukow-Levitzow',
 'Schloßweg 8, 17168 Lelkendorf',
 'Dorfstraße 8A, 17168 Thürkow',
 'Schlossallee 1, 17168 Groß Wüstenfelde',
 'Parkweg 13, 17168 Thürkow',
 'Hauptstraße 12, 17168 Prebberede',
 'Dörpstrat 3, 17168 Warnkenhagen',
 'Remlin 28, 17168 Schwasdorf']

In [78]:
import pandas as pd

df_locations = pd.DataFrame({"Address": addresses,
                             "Latitude": df["latitude"],
                             "Longitude": df["longitude"]
                             })

df_locations.head(10)


,Address,Latitude,Longitude
6251,"Dorfstraße 28, 17166 Groß-Wokern",53.7513,12.4925
6252,"Gutshof 10, 17166 Schorssow",53.7123,12.6257
6253,"Hauptstraße 3, 17166 Dalkendorf",53.8148,12.5021
6254,"Seestraße 4, 17166 Schorssow",53.6899,12.6007
6255,"Dorfstraße 16, 17166 Alt Sührkow",53.8024,12.6512
6256,"Dorfstraße 22, 17166 Hohen-Demzin",53.7214,12.5438
6257,"Nördliche Ringstraße 41, 17166 Teterow",53.7754,12.5749
6258,"Pösel 17, 17166 Groß Roge",53.7785,12.5047
6259,"Philosophenweg 2, 17166 Dahmen",53.6609,12.5814
6260,"Alte Dorfstraße 2, 17168 Jördenstorf",53.8782,12.6162


In [79]:
df_locations.to_pickle("data/locations.pkl")


In [80]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

categories = [["4d4b7105d754a06374d81259", "food", "red"], ["4f4528bc4b90abdf24c9de85", "sports", "green"], ["4bf58dd8d48988d124941735", "office", "black"], ["4c38df4de52ce0d596b336e1", "parking", "blue"]]

def get_categories(categories):
    return [(cat["name"], cat["id"]) for cat in categories]

def format_address(location):
    address = ", ".join(location["formattedAddress"])
    address = address.replace(", Deutschland", "")
    address = address.replace(", Germany", "")
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = "20210810"  # 20180724 - YYYYMMDD
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}".format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        venues = [(item["venue"]["id"],
                   item["venue"]["name"],
                   get_categories(item["venue"]["categories"]),
                   (item["venue"]["location"]["lat"], item["venue"]["location"]["lng"]),
                   format_address(item["venue"]["location"]),
                   item["venue"]["location"]["distance"]) for item in results]        
    except:
        venues = []
    return venues


def get_all_venues_near_location(lat, lon, client_id, client_secret, radius=500, limit=100):
    version = "20210810"  # 20180724 - YYYYMMDD
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        client_id, client_secret, version, lat, lon, radius, limit)
    try:
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        venues = [(item["venue"]["id"],
                   item["venue"]["name"],
                   get_categories(item["venue"]["categories"]),
                   (item["venue"]["location"]["lat"],
                    item["venue"]["location"]["lng"]),
                   format_address(item["venue"]["location"]),
                   item["venue"]["location"]["distance"]) for item in results]
    except:
        venues = []
    return venues


In [81]:
# Let"s now go over our neighborhood locations and get nearby restaurants; we"ll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

from constants import FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET

def get_info(lats, lons, **kwargs):
    _venues = []

    for lat, lon in zip(lats, lons):
        if kwargs:
            # Using radius=350 to meke sure we have overlaps/full coverage so we don"t miss any restaurant (we"re using dictionaries to remove any duplicates resulting from area overlaps)
            venues = get_venues_near_location(
                lat, lon, kwargs.id, FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET, radius=8000, limit=200)
            _venues.append(venues)
            print(" .", end="")
        else:
            venues = get_all_venues_near_location(lat, lon, FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET, radius=8000, limit=200)
            for venue in venues:
                venue_id = venue[0]
                venue_name = venue[1]
                venue_latlon = venue[3]
                venue_address = venue[4]
                venue_distance = venue[5]
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                venues = (
                    venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y)
            _venues.append(venues)
            print(" .", end="")
    print(" done.")
    return _venues


# Try to load from local file system in case we did this before
venues_all = {}
venues = {}

try:
    with open(f"data/venues_all.pkl", "rb") as f:
        venues_all = pickle.load(f)
    print(f"venues_all data loaded.")
except:
    pass

if not venues_all:
    print(
        f"Obtaining all venues around candidate locations:", end="")
    venues_all = get_info(df["latitude"], df["longitude"])
    print(venues_all)

    # Let"s persists this in local file system
    with open(f"data/venues_all.pkl", "wb") as f:
        pickle.dump(venues_all, f)


for id, cat, _ in categories:
    try:
        with open(f"data/venues_{cat}.pkl", "rb") as f:
            venues[cat] = pickle.load(f)
    except:
        pass

    if cat not in venues: 
        print(
            f"Obtaining {cat} venues around candidate locations:", end="")
        venues[cat] = get_info(df["latitude"], df["longitude"], id)

        # Let"s persists this in local file system
        with open(f"data/venues_{cat}.pkl", "wb") as f:
            pickle.dump(venues[cat], f)        
    else:
        if len(venues[cat]) > 0:
            print(f"venues_{cat} data loaded.")
        
            


Obtaining all venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.
[('4d20a665bdd7a093291eefce', 'Star Tankstelle', 53.75894928, 12.37820187, 'Zu den Wiesen 1, 18279 Lalendorf', 7570, 327169.1466285362, 5959893.009967239), ('59f31680535d6f6eee65d663', 'Auto Dienst Schmidt GmbH & Co. KG', 53.7553313, 12.7021209, 'Rempliner Hauptstr. 5, 17139 Remplin', 6947, 348505.41640348185, 5958750.877684891), ('4e11dd171495c8d31bca0d42', 'famila', 53.78742201797532, 12.570994234875146, 'Rostocker Str. 162, 17166 Teterow', 5459, 339984.1486403192, 5962607.916060259), ('4c793a66df08a1cd09ded95d', 'Gut Ulrichshusen', 53.62695267760596, 12.625961648056917, 'Seestraße, 17194 Ulrichshusen', 72

In [85]:
map_rostock = folium.Map(location=rostock_center, zoom_start=10)

folium.Marker(rostock_center, popup="Rostock").add_to(map_rostock)
for v in venues_all:
    folium.CircleMarker([v[2], v[3]], radius=3, color="blue", fill=True,
                        fill_color="blue", fill_opacity=1).add_to(map_rostock)
map_rostock


In [87]:
map_rostock = folium.Map(location=rostock_center, zoom_start=10)

folium.Marker(rostock_center, popup="Rostock").add_to(map_rostock)
for _, cat, color in categories:
    for v in venues[cat]:
        folium.CircleMarker(v[3], radius=3, color=color, fill=True,
                            fill_color=color, fill_opacity=1).add_to(map_rostock)

map_rostock
